# 概要

こんな人向け：コンペ初心者。何したらいいかわからない人。

自分の備忘録も兼ねているので間違っていたら教えてください。

# 1. LOADING

まずはデータを読み込みましょう。

In [ ]:
import numpy as np
import pandas as pd

numpy：各種計算に使用

pandas：表計算やデータのロードに使用

両方ともほぼ必ず使用するのでとりあえずインポートしてもいいくらいです。

In [ ]:
train = pd.read_csv("../input/bms-molecular-translation/train_labels.csv")
print(train.shape)
train.head()

pandasの.read_csvでファイルを読み込みます。

右端の|<というメニューからパスをコピーできます。

242万行のデータですね。

image_idは画像データのＩＤでInChIが今回予測したい文字列です。

# 2. IMAGE

In [ ]:
print("../input/bms-molecular-translation/train/0/0/0/000011a64c74.png")

画像はこんな感じでimage_idの1,2,3文字目でフォルダ分けされ、最後にimage_id.pngの名前で保存されています。

In [ ]:
import cv2

cv2は画像処理によく使われます。

In [ ]:
train["file_path"] = train["image_id"].apply(lambda x: f"../input/bms-molecular-translation/train/{x[0]}/{x[1]}/{x[2]}/{x}.png")
print(train["file_path"].values[0])

image_idを.applyで処理していきます。１文字目がx[0],２文字目がx[1]...となりパスが作成できます。

In [ ]:
path = train["file_path"].values[0]
image = cv2.imread(path)
print(image.shape)

cv2.imreadで指定したパスの画像を読み込みました。

この画像は229x325で赤緑青３色です。

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("seaborn-white")
plt.imshow(image)
plt.show()

matplotlibはグラフ作成によく使われるライブラリですが、画像表示にも利用できます。

有機物の構造式ですね。

In [ ]:
print(train["InChI"].values[0])

これが上図構造式のInChI表示です。画像からInChIを予測するのが今回のタスクです。

# 3. VOCABULARY

文字を予測しなければならないので、どんな文字があるのか整理します。

In [ ]:
words = set()
for s in train["InChI"]:
    words.update(set(s))
print(words)

set()はユニークなデータを取り出すことができます。

データのInChIすべてに対してユニークな文字を取り出し、wordsを更新しましょう。

全行の処理が終わったころにはすべてのInChIに含まれる文字がwordsに入っています。

In [ ]:
vocab = list(words)
print(vocab)

このvocabリストにある文字を使えばデータ内にあるすべてのInChIを表現できます。

In [ ]:
vocab.append("<sos>")
vocab.append("<eos>")
vocab.append("<pad>")
print(vocab)

ここで３つの文字を追加しましょう。

sos：文字の始まり。

eos：文字の終わり。

pad：パディング。データサイズの調整。

後で役に立ちます。

In [ ]:
VOCAB_SIZE = len(vocab)
print(VOCAB_SIZE)

これで文字を整理する一連の作業は終わり。合計で41種の文字を利用します。

In [ ]:
stoi = {
    'C': 0,')': 1,'P': 2,'l': 3,'=': 4,'3': 5,
    'N': 6,'I': 7,'2': 8,'6': 9,'H': 10,
    '4': 11,'F': 12,'0': 13,'1': 14,'-': 15,
    'O': 16,'8': 17,',': 18,'B': 19,'(': 20,
    '7': 21,'r': 22,'/': 23,'m': 24,'c': 25,
    's': 26,'h': 27,'i': 28,'t': 29,'T': 30,
    'n': 31,'5': 32,'+': 33,'b': 34,'9': 35,
    'D': 36,'S': 37,'<sos>': 38,'<eos>': 39,'<pad>': 40
}

itos = {item[1]:item[0] for item in stoi.items()}

機械学習に文字を直接ぶち込むことはできません。

なので文字を数値に変換して学習と予測を実行し、最後にまた文字に戻す作業が必要です。

stoiはstring to index(indices)でもじを数値に変えます。itosは逆。

In [ ]:
inchi = train["InChI"].values[0]
print(inchi)
inchi = [stoi[s] for s in inchi]
print(inchi)

このように文字を数値のリストに変換できます。

In [ ]:
train["length"] = train["InChI"].apply(lambda x: len(x))
MAX_LEN = train["length"].max()
print(MAX_LEN)
train.head()

次に最大文字数を取得しておきましょう。これも後々必要です。

In [ ]:
path = train.loc[train["length"] == MAX_LEN]["file_path"].values[0]
plt.imshow(cv2.imread(path))
plt.show()
print(train.loc[train["length"] == MAX_LEN]["InChI"].values[0])

ちなみに最長の構造はこれです。こんなもの予測できたら苦労しませんね。

# 4. DATASET

今回モデル作成にはpytorchを使います。

pytorchではモデルに読み込ませるデータを自作することが多いです。

とりあえず最低限必要なデータは画像とInChIです。

まずはどんな手順でデータを作成するか確認しましょう。

## ◆◇IMAGE◇◆
画像はさっきと同じ要領

In [ ]:
path = train["file_path"].values[0]
image = cv2.imread(path)
print(image.shape)

さっきと同じ。

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

cv2で画像を読み込むとBGRという色の順になります。

これをRGBに直すために.COLOR_BGR2RGBに通しました。

これが学習に効くのかは知りません。私は気分だと思っています。

In [ ]:
path = train["file_path"].values[1]
image = cv2.imread(path)
print(image.shape)

ここで気を付けるべきは画像サイズ。

１枚目は229x325でしたが２枚目は148x288です。

画像サイズを統一しないとエラーになるので処理しましょう。

In [ ]:
import albumentations as A

albumentationsは画像加工に便利なライブラリです。

In [ ]:
path = train["file_path"].values[0]
image = cv2.imread(path)
image = A.Resize(256, 256)(image = image)
print(image)

.Resizeで256x256に変換しました。

注意点は辞書型で返ってくることです。

In [ ]:
image = image["image"]
print(image.shape)

変換されていますね。

In [ ]:
image = A.Normalize()(image = image)["image"]
print(image)

次は正規化処理です。これをしておくと学習が早く収束する？みたいです。

In [ ]:
from albumentations.pytorch import ToTensorV2
image = ToTensorV2()(image = image)["image"]
print(image.shape)

albumentations.pytorch.ToTensorV2でtorch型に変換しましょう。

pytorchでは色を表すデータを１個目に使うので3x256x256となっています。

画像処理はこんな感じです。

他にもalbumentationsで左右反転や回転などさせたりしますが、割愛。

## ◆◇InChI◇◆
次は予測したいInChIの加工方法です。

In [ ]:
inchi = train["InChI"].values[0]
print(inchi)
inchi = [stoi[s] for s in inchi]
print(inchi)

これでリスト化までできました。

In [ ]:
inchi.insert(0, stoi["<sos>"])
inchi.append(stoi["<eos>"])
print(inchi)

文字の始まりsosと終わりeosを足しておきましょう。

In [ ]:
import torch
inchi = torch.LongTensor(inchi)
print(inchi)

最後にpytorchの.LongTensorでpytoch用のデータに変換します。

これで終わり。

## ◆◇DATASET◇◆
これまでの処理をまとめてデータセットにします。

In [ ]:
from torch.utils.data import Dataset, DataLoader

Datasetでセットを作り、DataLoaderで読み込みます。

こいつらはほぼ必須なので覚えておきましょう。

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.paths = self.df["file_path"].values
        self.inchi = self.df["InChI"].values
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = A.Compose([
            A.Resize(256, 256),
            A.Normalize(),
            ToTensorV2()
        ])(image = image)["image"]
        inchi = self.inchi[idx]
        inchi = [stoi[s] for s in inchi]
        inchi.insert(0, stoi["<sos>"])
        inchi.append(stoi["<eos>"])
        inchi = torch.LongTensor(inchi)
        return image, inchi

データセットはクラスで作ります。

init：初期化条件。dfは後々渡すデータになります。

len：データサイズを定義するためのもの。大抵はデータの行数にします。

getitem：実際にデータを呼び出すための関数。引数にはインデックスが使われます。

最初はよくわからないかもしれませんが３回くらい自作してみると理解できます。

getitemではさっき書いた処理を全て行いましょう。

idxでインデックスが渡されるので、パスやInChIを読み取って変換していきます。

albumentationsの.Composeを使えば一連の処理を実行してくれるので便利です。

最後にreturnで画像とinchiを出しましょう。

In [ ]:
ds = TrainDataset(train)

実際にデータセットを作りました。

initで定義した引数を渡します。今回はデータフレームのみ。

In [ ]:
print(ds[0])

１個目のデータをとりだしました。

画像データとInChIが出力されます。

In [ ]:
print(len(ds[0]))
print(ds[0][0].shape, ds[0][1].shape)

サイズもあってますね。

In [ ]:
for i in range(5):
    print(ds[i][0].shape, ds[i][1].shape)

ここでネックなのがInChIの文字数が統一されていないことです。

これからDataLoaderでバッチ(まとまり)ごとに取り出す変換を行います。

その際に画像と同じくInChIのサイズもバッチ内で統一しないといけません。

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def bms_collate(batch):
    images, labels = [], []
    for data in batch:
        images.append(data[0])
        labels.append(data[1])
    labels = pad_sequence(labels, batch_first = True, padding_value = stoi["<pad>"])
    return torch.stack(images), labels

toch.nn.utils.rnnにpad_sequenceという関数があります。

これはtensor型のデータをパディングしサイズを統一してくれます。

パディングにつかう文字は"pad"です。ここで役に立ってくれるわけです。

In [ ]:
loader = DataLoader(ds, batch_size = 8, collate_fn = bms_collate)

DataLoaderにデータセットとバッチサイズ(１度に何個取り出すか)、さっきの変換関数を渡します。

In [ ]:
batch = next(iter(loader))
print(batch[0].shape, batch[1].shape)

next(iter())で次のバッチを取り出せます。バッチサイズを見てみると８になっていますね。

In [ ]:
for i in range(8):
    print(batch[1][i])
    print("=" * 100)

InChIを確かめましょう。

全ての文字が"sos"の38からスタートしてeosの39で終わり、残り文字数は"pad"の40で調整されています。

１個だけ39で終わっているデータがあるので、そいつの文字数に合わせてパディングされているわけです。

# 5. CV
データ分割はめんどくさいのでtrain_test_splitにします。

本来はもっと正確な分割をすべきです。

In [ ]:
DEBUG = True
if DEBUG:
    df = train.sort_values(by = "length").reset_index(drop = True).copy()
    df = df.iloc[:1000, :]
else:
    df = train.copy()
print(df.shape)

上手くモデルが作成できているか確認するだけならデータサイズを落としましょう。

簡単にしたいのでInChIの文字数が少ないデータにしました。

実際に提出する際はDEBUGをFalseにしますが、１回の学習に６時間以上かかるので学習コストはかなり厳しいです。

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_valid = train_test_split(df, test_size = 0.1, shuffle = True)
print(df_train.shape, df_valid.shape)

データ分割しました。学習が900行で評価が100行です。

In [ ]:
from tqdm.notebook import tqdm

train_data = TrainDataset(df_train)
valid_data = TrainDataset(df_valid)
train_loader = DataLoader(train_data, batch_size = 64, shuffle = True, drop_last = True, collate_fn = bms_collate, num_workers = 4)
valid_loader = DataLoader(valid_data, batch_size = 64 * 2, shuffle = False, drop_last = False, collate_fn = bms_collate, num_workers = 4)

for batch in tqdm(train_loader):
    pass
for batch in tqdm(valid_loader):
    pass

先程と同じ要領でデータセットとデータローダーを作ります。

shuffleはデータシャッフル。drop_lastはバッチで切ったデータの最後の端数を切るかどうか。

num_workersは並行処理数です。後でわかりますが計算量が膨大なのでGPUを使うことになります。その為に処理数を増やしておきます。

tqdmを使うと進捗がわかりやすいので便利です。とりあえずpassで全データを読み込めているか確認しました。

# 6. MODEL

In [ ]:
import torch.nn as nn
import torchvision

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

torch.nnとtorchvisionをインポートします。

DEVICEはCPUorGPUを定義します。

GPUがONになっていると.cuda.is_availableはTrueです。

## ◆◇Encoder◇◆

今回画像データから特徴量を作成するモデルをEncoderとします。

In [ ]:
resnet = torchvision.models.resnet18()
for params in resnet.state_dict():
    print(params)

torchvisionからresnet18を取り出しました。

In [ ]:
modules = list(resnet.children())[:-2]
resnet = nn.Sequential(*modules)
for params in resnet.state_dict():
    print(params)

最後のfc層は不要なので消しておきましょう。

In [ ]:
batch = next(iter(train_loader))
image = batch[0][0]
print(image.shape)

試しに１つ画像を通します。

In [ ]:
encoded = resnet(image.unsqueeze(0))
print(encoded.shape)

変換後のサイズは512x8x8です。

こんな感じでresnetを使って画像の特徴をとらえたデータを作成しましょう。

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        modules = list(resnet.children())[:-2]
        self.resnet = nn.Sequential(*modules)
        
    def forward(self, images):
        batch_size = images.size(0)
        features = self.resnet(images)
        features = features.permute(0, 2, 3, 1)
        features = features.view(features.size(0), -1, features.size(-1))
        return features
    
encoder = Encoder()
encoded = encoder(batch[0])
print(encoded.shape)

クラスでEncoderを作りました。

initではresnetを定義します。

forwardでは実際に行われる処理を記述します。今回は画像データをresnetにとおして出力しましょう。

.permuteはデータの行列(位置)を変える関数です。512を最後に持っていきます。

次元数が多いと不便なので.viewで調整しています。-1の部分は任意の数になるので8x8になります。

## ◆◇Decoder◇◆

DecoderはEncoderでの変換後の画像データとInChIを学習させるモデルです。

LSTMを使うと時系列に対応した学習が可能です。

In [ ]:
inchi = batch[1]
print(inchi.shape)
print(inchi)

InChIはこのようにインデックスのリストにしました。

In [ ]:
embedding = nn.Embedding(num_embeddings = VOCAB_SIZE, embedding_dim = 256)
embeds = embedding(inchi)
print(embeds.shape)
print(embeds)

Embeddingは文字を数値ベクトルに変換します。

引数は語彙数(今回はVOCAB_SIZE)と変換後の次元数（適当）です。存在を忘れていたVOCAB_SIZEがここで使われます。

変換後のサイズを見ると256にデータが拡張されています。

In [ ]:
print(encoded.shape)
encoded = encoded.mean(dim = 1)
print(encoded.shape)

次に画像データの特徴を使いますが、次元数が多いのでdim = 1において平均をとりました。

平均にした深い意味はありません。サイズ調整のためだけです。

In [ ]:
print(inchi[0])

これからLSTMで時系列データを学習させます。

例えば１個目のInChIは38, 7, 31...と続いていますが、最初の38と画像の特徴から次の７を予測したいわけです。

In [ ]:
print(embeds[:, 0].shape)
print(encoded.shape)

１文字目はembedsの１列目に該当します。

そして画像の特徴はEncoderで変換して.meanでサイズ調整したデータです。

これらから２文字目の７を学習させます。

In [ ]:
lstm = nn.LSTMCell(input_size = (256 + 512), hidden_size = 512, bias = True)

LSTMCellはLSTMと呼ばれる時系列データが学習可能な層です。

詳細は詳しく語れないので調べてください。

input_sizeに入力するサイズを指定します。今回は１つの文字をEmbeddingで変換した256と画像の特徴である512を足したサイズになります。

LSTMに渡すデータは１つ前の文字と画像の特徴、そして１つ前のLSTMが出力した戻り値(h, c)です。

In [ ]:
init_h = nn.Linear(in_features = 512, out_features = 512)(encoded)
init_c = nn.Linear(in_features = 512, out_features = 512)(encoded)
print(init_h.shape, init_c.shape)

最初の文字は前の文字のデータがないので全結合層でh,cを作ります。

In [ ]:
lstm_input = torch.cat((embeds[:, 0], encoded), dim = 1)
print(lstm_input.shape)

文字データと画像の特徴量は.catでくっつけました。

In [ ]:
h, c = lstm(lstm_input, (init_h, init_c))
print(h.shape, c.shape)

これらをLSTMに渡しましょう。出力されるのは予測した文字に当たるhと記憶状態を表すCです。

意味不明かもしれませんが、私は「へー」くらいの理解にしています。

In [ ]:
lstm_input = torch.cat((embeds[:, 1], encoded), dim = 1)
h, c = lstm(lstm_input, (h, c))
print(h.shape, c.shape)

３文字目は２文字目とさっきの戻り値h,cとで入力します。

これを全文字数繰り返すことでInChIを学習していくわけです。

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, encoder_dim, decoder_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.decoder_dim = decoder_dim
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.init_h = nn.Linear(in_features = encoder_dim, out_features = decoder_dim)
        self.init_c = nn.Linear(in_features = encoder_dim, out_features = decoder_dim)
        self.lstm = nn.LSTMCell(input_size = (embed_size + encoder_dim), hidden_size = decoder_dim, bias = True)
        self.drop = nn.Dropout(p = 0.3)
        self.linear = nn.Linear(in_features = decoder_dim, out_features = vocab_size)
        
    def forward(self, features, inchis): #渡されるのは画像特徴量と実際のInChI
        embeds = self.embedding(inchis)
        
        features = features.mean(dim = 1)
        h = self.init_h(features) #最初は画像だけからh,cを作る
        c = self.init_c(features)
        
        seq_length = len(inchis[0]) - 1 #最後の文字になると終わりなので１だけ引く
        batch_size = inchis.size(0)
        preds = torch.zeros(batch_size, seq_length, self.vocab_size).to(DEVICE) #予測した文字を格納するために作成
        
        for s in range(seq_length):
            lstm_input = torch.cat((embeds[:, s], features), dim = 1) #s文字目と画像特徴量をくっつける
            h, c = self.lstm(lstm_input, (h, c)) #hが予測されたデータ
            x = self.drop(h)
            x = self.linear(x)
            preds[:, s] = x #s文字目の次の文字の予測
        return preds

Decoderをクラスで作成しました。

やってることはこれまでの解説と同じです。

最後に予測したpredsを吐き出したいのでバッチサイズと文字数、VOCAB_SIZEで枠を作りましょう。

In [ ]:
print(batch[0].shape, batch[1].shape)
encoded = encoder(batch[0])
print(encoded.shape)

振り出しに戻りますがバッチデータの画像は3x256x256でInChIが54各文字です。

Encoderで画像から特徴量を取り出します。

これらがDecoderの入力となります。

In [ ]:
decoder = Decoder(vocab_size = VOCAB_SIZE, embed_size = 256, encoder_dim = 512, decoder_dim = 512)
preds = decoder(encoded, batch[1])
print(preds.shape)

Decoderに入れるとバッチサイズ x 文字数-1 x 語彙数で出力されました。

In [ ]:
print(preds[0].shape)
print(preds[0])

こんな感じで各文字に対して各語彙の数値が格納されています。

41文字の中からもっとも値の大きい文字を予測した文字としましょう。

# 7. TRAINING

In [ ]:
if DEBUG:
    BATCH_SIZE = 16
    EPOCHS = 5
else:
    BATCH_SIZE = 64
    EPOCHS = 1

In [ ]:
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle = True, drop_last = True, collate_fn = bms_collate, num_workers = 4)
valid_loader = DataLoader(valid_data, batch_size = BATCH_SIZE * 2, shuffle = False, drop_last = False, collate_fn = bms_collate, num_workers = 4)

encoder = Encoder().to(DEVICE)
decoder = Decoder(vocab_size = VOCAB_SIZE, embed_size = 256, encoder_dim = 512, decoder_dim = 512).to(DEVICE)

encoder_optimizer = torch.optim.Adam( encoder.parameters(), lr = 1e-4, weight_decay = 1e-6, amsgrad = False)
decoder_optimizer = torch.optim.Adam( decoder.parameters(), lr = 1e-4, weight_decay = 1e-6, amsgrad = False)

encoder_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR( encoder_optimizer, T_max = 4, eta_min = 1e-4, last_epoch = -1 )
decoder_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR( decoder_optimizer, T_max = 4, eta_min = 1e-4, last_epoch = -1 )

criterion = nn.CrossEntropyLoss(ignore_index = stoi["<pad>"])

optimizer：最適化手法。とりあえずAdam

scheduler：学習率(lr)の調整。色々あるけどみんな使ってるCosineAnnealingLR。

criterion：損失関数。CrossEntropyLossはsoftmaxに変換してから誤差を計算してくれます。"pad"は学習に関係ないので除外しましょう。

In [ ]:
encoder.train()
decoder.train()
for images, inchis in tqdm(train_loader):
    images = images.to(DEVICE)
    inchis = inchis.to(DEVICE)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    encoded = encoder(images)
    preds = decoder(encoded, inchis)
    print(preds.shape, inchis.shape)

学習させるときは.train()で学習モードにします。何をかえているかは知りません。

各変換を経てpredsを出力しましょう。

ここでサイズを見てみるとpredsは53文字に対して実際のInChIは"sos"があるので54文字です。

なのでlossが計算できません。

In [ ]:
loss = criterion(preds.permute(0, 2, 1), inchis[:, 1:])
print(loss.item())

InChIは"sos"がいらないので１以降にしましょう。

またpredsは現在の順番ではlossが計算できないので入れ替えました。（ココはあってるか怪しいです。。。）

In [ ]:
encoder.train()
decoder.train()
for images, inchis in tqdm(train_loader):
    images = images.to(DEVICE)
    inchis = inchis.to(DEVICE)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    encoded = encoder(images)
    preds = decoder(encoded, inchis)
    loss = criterion(preds.permute(0, 2, 1), inchis[:, 1:])
    print(loss.item())
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

lossを計算できたらloss.backwardとoptimizer.stepで更新しましょう。

この辺りはpytorchの基礎を検索するとわかりやすい記事があります。

以上で１回の学習が終わりました。

In [ ]:
encoder.eval()
decoder.eval()
valid_loss = 0
for images, inchis in tqdm(valid_loader):
    images = images.to(DEVICE)
    inchis = inchis.to(DEVICE)
    with torch.no_grad():
        encoded = encoder(images)
        preds = decoder(encoded, inchis)
        loss = criterion(preds.permute(0, 2, 1), inchis[:, 1:])
        print(loss.item())
        valid_loss += loss.item()
valid_loss /= len(valid_loader)
print("mean loss : ", valid_loss)

評価用データのvalidでlossを計算しましょう。これが実際に予測したときに近い誤差です。

このlossが小さければ小さいほど（今回のデータ分割において）良いモデルです。

In [ ]:
best_loss = np.inf
for epoch in range(EPOCHS):
    encoder.train()
    decoder.train()
    for images, inchis in tqdm(train_loader):
        images = images.to(DEVICE)
        inchis = inchis.to(DEVICE)
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        encoded = encoder(images)
        preds = decoder(encoded, inchis)
        loss = criterion(preds.permute(0, 2, 1), inchis[:, 1:])
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
    encoder.eval()
    decoder.eval()
    valid_loss = 0
    for images, inchis in tqdm(valid_loader):
        images = images.to(DEVICE)
        inchis = inchis.to(DEVICE)
        with torch.no_grad():
            encoded = encoder(images)
            preds = decoder(encoded, inchis)
            loss = criterion(preds.permute(0, 2, 1), inchis[:, 1:])
            valid_loss += loss.item()
    valid_loss /= len(valid_loader)
    print(f"[epoch{epoch}] loss:{valid_loss}")
    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(encoder.state_dict(), "bms_encoder.pth")
        torch.save(decoder.state_dict(), "bms_decoder.pth")
        print("saved...")

EPOCHの数だけ学習を繰り返しました。

学習を重ねる度にlossが減りますが必ず減るわけではありません。

なのでbest_lossを無限大で定義して更新していき、最もlossの小さいモデルを保存しましょう。

# 8. INFERENCE
最後に画像データのみから文字を作成しなければなりません。

そのためにDecoderにpredict関数を足します。

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, encoder_dim, decoder_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.decoder_dim = decoder_dim
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.init_h = nn.Linear(in_features = encoder_dim, out_features = decoder_dim)
        self.init_c = nn.Linear(in_features = encoder_dim, out_features = decoder_dim)
        self.lstm = nn.LSTMCell(input_size = (embed_size + encoder_dim), hidden_size = decoder_dim, bias = True)
        self.drop = nn.Dropout(p = 0.3)
        self.linear = nn.Linear(in_features = decoder_dim, out_features = vocab_size)
        
    def forward(self, features, inchis): #渡されるのは画像特徴量と実際のInChI
        embeds = self.embedding(inchis)
        
        features = features.mean(dim = 1)
        h = self.init_h(features) #最初は画像だけからh,cを作る
        c = self.init_c(features)
        
        seq_length = len(inchis[0]) - 1 #最後の文字になると終わりなので１だけ引く
        batch_size = inchis.size(0)
        preds = torch.zeros(batch_size, seq_length, self.vocab_size).to(DEVICE) #予測した文字を格納するために作成
        
        for s in range(seq_length):
            lstm_input = torch.cat((embeds[:, s], features), dim = 1) #s文字目と画像特徴量をくっつける
            h, c = self.lstm(lstm_input, (h, c)) #hが予測されたデータ
            x = self.drop(h)
            x = self.linear(x)
            preds[:, s] = x #s文字目の次の文字の予測
        return preds

    def predict(self, features, max_len): #画像特徴量、最大文字数
        batch_size = features.size(0)
        features = features.mean(dim = 1)
        h = self.init_h(features)
        c = self.init_c(features)
        
        word = torch.full((batch_size, 1), stoi["<sos>"]).to(DEVICE) #1文字目は必ず"sos"
        embeds = self.embedding(word)
        preds = torch.zeros((batch_size, max_len), dtype = torch.long).to(DEVICE) #出力する予測の枠
        preds[:, 0] = word.squeeze() #１文字目の"sos"を入れておくだけ
        for i in range(max_len): #最大文字数まで予測を繰り返す
            lstm_input = torch.cat((embeds[:, 0], features), dim = 1) #１つ前の文字と画像特徴量をくっつける
            h, c = self.lstm(lstm_input, (h, c))
            x = self.drop(h)
            x = self.linear(x)
            x = x.view(batch_size, -1)
            pred_idx = x.argmax(dim = 1) #最も値の大きい語彙を予測とする
            preds[:, i] = pred_idx #出力データに格納
            embeds = self.embedding(pred_idx).unsqueeze(1) #次の文字の予測に使用する
        return preds

predict関数を作りました。

１つ前の文字をEmbeddingで変換して画像特徴量とh,cから次の文字hを出力する処理を最大文字数まで繰り返しましょう。

In [ ]:
encoder = Encoder().to(DEVICE)
decoder = Decoder(vocab_size = VOCAB_SIZE, embed_size = 256, encoder_dim = 512, decoder_dim = 512).to(DEVICE)
encoder.load_state_dict(torch.load("./bms_encoder.pth", map_location = DEVICE))
decoder.load_state_dict(torch.load("./bms_decoder.pth", map_location = DEVICE))

最もlossの小さいモデルを保存しているので取り出します。

In [ ]:
encoder.eval()
decoder.eval()
preds = []
for images, inchis in tqdm(valid_loader):
    images = images.to(DEVICE)
    with torch.no_grad():
        encoded = encoder(images)
        pred = decoder.predict(encoded, max_len = MAX_LEN)
        preds.append(pred)

評価用データで予測してみます。

ここでMAX_LENが使われます。

In [ ]:
print(preds[0].shape)
preds[0][0]

予測したラベルを見てみましょう。よくわかりませんね。

In [ ]:
def generate_inchi(pred):
    label = [itos[i] for i in pred.to("cpu").numpy()]
    result = []
    for i in range(len(label)):
        if label[i] == "<eos>":
            break
        result.append(label[i])
    result = "".join(result)
    return result

result = generate_inchi(preds[0][0])
print(result)
print(df_valid["InChI"].values[0])

"eos"になるまで文字を取り出す関数を作ります。

これに予測ラベルを入れるとInChIになります。

実際の文字と比べてみるとまぁまぁかなといったところ。

# 9. SUBMIT

In [ ]:
test = pd.read_csv("../input/bms-molecular-translation/sample_submission.csv")
test["file_path"] = test["image_id"].apply(
    lambda x: f"../input/bms-molecular-translation/test/{x[0]}/{x[1]}/{x[2]}/{x}.png"
)
print(test.shape)
test.head()

提出するtestデータのパスを作成します。

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.paths = self.df["file_path"].values
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = A.Compose([
            A.Resize(256, 256),
            A.Normalize(),
            ToTensorV2()
        ])(image = image)["image"]
        return image
    
#test_data = TestDataset(test.iloc[:200, :])
test_data = TestDataset(test)
test_loader = DataLoader(test_data, batch_size = BATCH_SIZE * 4, shuffle = False, drop_last = False, num_workers = 4)

testではInChIがない点だけ注意しましょう。collate_fnも不要です。

In [ ]:
preds = []
for images in tqdm(test_loader):
    with torch.no_grad():
        images = images.to(DEVICE)
        encoded = encoder(images)
        pred = decoder.predict(encoded, max_len = MAX_LEN)
        preds.append(pred)

testから提出用の予測データを作ります。

CPUだと絶望的に長いのでGPUを使いましょう。

In [ ]:
generate_inchi(preds[0][0])

予測はこんな感じ。

In [ ]:
submit_preds = []
for pred in preds:
    submit_preds.append([generate_inchi(p) for p in pred])
submit_preds = np.concatenate(submit_preds, axis = 0)
print(submit_preds.shape)

各予測をまとめます。

In [ ]:
#submit = test[["image_id", "InChI"]].iloc[:200, :].copy()
submit = test[["image_id", "InChI"]].copy()
submit["InChI"] = submit_preds
submit.to_csv("submission.csv", index = False)
submit

testから"image_id"と"InChI"を頂戴してさっきの予測データを"InChI"に入れましょう。

これで提出ができます。